## Factorization Machines with tensorflow tutorial

In this tutial we demonstrate how to create a FMs model with tensorflow step-by-step.

### References:

Blog post by Gabriele Modena: [Factorization Machines with Tensorflow](http://nowave.it/factorization-machines-with-tensorflow.html)

Factorization Machines paper: [Factorization Machines with LibFm (pdf)](http://www.csie.ntu.edu.tw/~b97053/paper/Factorization%20Machines%20with%20libFM.pdf)


### Relevant repository:

[tffm library](https://github.com/geffy/tffm)

## Utility function to convert list to sparse matrix

Here we created a utility function to create a sparse matrix (that is needed by factorization machines) from a list of user/item ids.

Check [this gist](https://gist.github.com/babakx/7a3fc9739b7778f6673a458605e18963) for more details about this utitly function.

In [11]:
from itertools import count
from collections import defaultdict
from scipy.sparse import csr

def vectorize_dic(dic, ix=None, p=None):
    """ 
    Creates a scipy csr matrix from a list of lists (each inner list is a set of values corresponding to a feature) 
    
    parameters:
    -----------
    dic -- dictionary of feature lists. Keys are the name of features
    ix -- index generator (default None)
    p -- dimension of featrure space (number of columns in the sparse matrix) (default None)
    """
    if (ix == None):
        ix = defaultdict(count(0).next)
        
    n = len(dic.values()[0]) # num samples
    g = len(dic.keys()) # num groups
    nz = n * g # number of non-zeros

    col_ix = np.empty(nz, dtype=int)
    
    i = 0
    for k, lis in dic.iteritems():
        # append index el with k in order to prevet mapping different columns with same id to same index
        col_ix[i::g] = [ix[str(el) + str(k)] for el in lis]
        i += 1
        
    row_ix = np.repeat(np.arange(0, n), g)
    data = np.ones(nz)
    
    if (p == None):
        p = len(ix)
        
    ixx = np.where(col_ix < p)

    return csr.csr_matrix((data[ixx],(row_ix[ixx], col_ix[ixx])), shape=(n, p)), ix

## Loading data

In this tutorial we use the [MovieLens100k Dataset](https://grouplens.org/datasets/movielens/100k/). Here we convert data to scipy csr (sparse) matrix format. 

In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer

# laod data with pandas
cols = ['user', 'item', 'rating', 'timestamp']
train = pd.read_csv('data/ua.base', delimiter='\t', names=cols)
test = pd.read_csv('data/ua.test', delimiter='\t', names=cols)

# vectorize data and convert them to csr matrix
X_train, ix = vectorize_dic({'users': train.user.values, 'items': train.item.values})
X_test, ix = vectorize_dic({'users': test.user.values, 'items': test.item.values}, ix, X_train.shape[1])
y_train = train.rating.values
y_test= test.rating.values

# print shape of data
print X_train.shape
print X_test.shape

(90570, 2623)
(9430, 2623)


## Define FM Model with tensorflow

We first initialize the parameters of the model as follows:

In [ ]:
import tensorflow as tf

n, p = X_train.shape

# number of latent factors
k = 10

# design matrix
X = tf.placeholder('float', shape=[None, p])
# target vector
y = tf.placeholder('float', shape=[None, 1])

# bias and weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([p]))

# interaction factors, randomly initialized 
V = tf.Variable(tf.random_normal([k, p], stddev=0.01))

# estimate of y, initialized to 0.
y_hat = tf.Variable(tf.zeros([n, 1]))

### Now we define how the output values y should be calculated
Using the trick in Rendle's paper, the output of a give feature vector `x` can be calculated using the following equation. The next cell implements that with tensorflow operations.

In [23]:
from IPython.display import display, Math, Latex
display(Math(r'\hat{y}(\mathbf{x}) = w_0 + \sum_{j=1}^{p}w_jx_j + \frac{1}{2} \sum_{f=1}^{k} ((\sum_{j=1}^{p}v_{j,f}x_j)^2-\sum_{j=1}^{p}v_{j,f}^2 x_j^2)'))

<IPython.core.display.Math object>

In [ ]:
# Calculate output with FM equation
linear_terms = tf.add(w0, tf.reduce_sum(tf.multiply(W, X), 1, keep_dims=True))
pair_interactions = (tf.multiply(0.5,
                    tf.reduce_sum(
                        tf.subtract(
                            tf.pow( tf.matmul(X, tf.transpose(V)), 2),
                            tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2)))),
                        1, keep_dims=True)))
y_hat = tf.add(linear_terms, interactions)

## Loss function

Here we implement FM point-wise loss function with tensorflow operations. The loss is defined as:

In [26]:
display(Math(r'L = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \lambda_w ||W||^2 + \lambda_v ||V||^2'))

<IPython.core.display.Math object>

In [ ]:
# L2 regularized sum of squares loss function over W and V
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_w, tf.pow(W, 2)),
                tf.multiply(lambda_v, tf.pow(V, 2)))))

error = tf.reduce_mean(tf.square(tf.subtract(y, y_hat)))
loss = tf.add(error, l2_norm)

## Optimization